# Analytics usando tipos de dados complexos
<p><img src = https://thumbs.dreamstime.com/b/cores-em-ascens%C3%A3o-superf%C3%ADcie-tridimensional-colorida-de-indiv%C3%ADduos-com-dados-complexos-arco-%C3%ADris-como-malha-arame-preto-ou-241163870.jpg width=500></p>

## Introdução
No módulo anterior, vimos como podemos importar e exportar dados para outras ferramentas analíticas para aproveitar as ferramentas analíticas fora do nosso banco de dados. Muitas vezes é mais fácil analisar números, mas no mundo real, os dados são frequentemente encontrados em outros formatos: palavras, locais, datas e, às vezes, estruturas de dados complexas. Neste módulo, veremos esses outros formatos e veremos como podemos usar esses dados na nossa análise.

Primeiro, veremos dois tipos de coluna comumente encontrados: colunas de data e hora (**datetime**) e colunas de **latitude** e **longitude**. Esses tipos de dados nos darão uma compreensão fundamental de como entender nossos dados de uma perspectiva temporal e geoespacial. Em seguida, veremos tipos de dados complexos, como **arrays** e **JSON**, e aprenderemos como extrair pontos de dados desses tipos de dados complexos. Essas estruturas de dados são frequentemente usadas para dados alternativos ou dados em nível de log, como logs de sites. Por fim, veremos como podemos extrair significado do texto em nosso banco de dados e usar dados de texto para extrair **insights**.
Ao final do módulo, você terá ampliado seus recursos de análise para que possa aproveitar praticamente qualquer tipo de dado disponível para você.
## Tipos de dados de data e hora para análise
Estamos todos familiarizados com datas e horários, mas não pensamos frequentemente em como essas medidas quantitativas são representadas. Sim, eles são representados por números, mas não por um único número. Em vez disso, eles são medidos com um conjunto de números, um para o ano, um para o mês, um para o dia do mês, um para a hora, um para o minuto e assim por diante.

O que podemos não perceber, porém, é que esta é uma representação complexa, composta por vários componentes diferentes. Por exemplo, saber o minuto atual sem saber a hora atual é inútil. Além disso, existem maneiras complexas de interagir com datas e horas, por exemplo, diferentes pontos no tempo podem ser subtraídos um do outro. Além disso, a hora atual pode ser representada de forma diferente dependendo de onde você está no mundo.

Como resultado desses meandros, precisamos ter um cuidado especial ao trabalhar com esse tipo de dados. Na verdade, o *Postgres*, como a maioria dos bancos de dados, oferece tipos de dados especiais que podem representar esses tipos de valores. Começaremos examinando o tipo de data (**date**).

### Começando com o tipo de data
As datas podem ser facilmente representadas usando strings, por exemplo, "1 de janeiro de 2000", que representa claramente uma data específica, mas as datas são uma forma especial de texto, pois representam um valor quantitativo e sequencial. Você pode adicionar uma semana à data atual, por exemplo. Uma determinada data tem muitas propriedades diferentes que você pode querer usar em sua análise, por exemplo, o ano ou o dia da semana que a data representa. Trabalhar com datas também é necessário para a análise de séries temporais, que é um dos tipos mais comuns de análise que surgem.

O padrão SQL inclui um tipo de dados **DATE** e o *PostgreSQL* oferece ótimas funcionalidades para interagir com esse tipo de dados. Primeiro, podemos configurar nosso banco de dados para exibir datas no formato com o qual estamos mais familiarizados. O PostgreSQL usa o parâmetro **DateStyle** para definir essas configurações. Para ver suas configurações atuais, você pode usar o seguinte comando:

`SHOW DateStyle;`

A seguir está a saída da consulta anterior:

In [80]:
# Importando o SqlAlchemy
import pandas as pd
from sqlalchemy import create_engine

cnxn_string = ("postgresql+psycopg2://{username}:{pswd}"
               "@{host}:{port}/{database}")

# Instanciando a Função create_engine com a string de conexão
engine = create_engine(cnxn_string.format(
    username="postgres",
    pswd="admin",
    host="127.0.0.1",
    port=5432,
    database="sqlda"))

# Criando a Consulta ou o comando SQL a ser executado
sql = """
		SHOW DateStyle
      """
# Conectando a Engine
conn = engine.connect()

# Executando o Comando SQL
rs = conn.execute(sql)

for row in rs:
    print(row)

('ISO, DMY',)


O primeiro parâmetro especifica o formato de saída da **International Organization Standardization (ISO)**, que exibe a data como Ano, Mês, Dia e o segundo parâmetro especifica a ordem (por exemplo, Mês, Dia, Ano versus Dia, Mês, Ano) para entrada ou resultado. Você pode configurar a saída para seu banco de dados usando o seguinte comando:

`SET DateStyle='ISO, MDY';`



In [87]:
sql = """
	  SET DateStyle='ISO, MDY';
      """
# Executando o Comando SQL
conn.execute(sql)

# Criando a Consulta ou o comando SQL a ser executado
sql1 = """
		SHOW DateStyle
      """

# Executando o Comando SQL
rs = conn.execute(sql1)

for row in rs:
    print(row)

('ISO, MDY',)


Por exemplo, se você quiser defini-lo para o formato europeu de Dia, Mês, Ano, defina DateStyle como 'GERMAN, DMY'. Para este módulo, usaremos o formato de exibição ISO (Ano, Mês, Dia) e o formato de entrada Mês, Dia, Ano. Você pode configurar esse formato usando o comando anterior.

In [82]:
sql = """
	  SET DateStyle='GERMAN, DMY';
      """
# Executando o Comando SQL
conn.execute(sql)

# Criando a Consulta ou o comando SQL a ser executado
sql1 = """
		SHOW DateStyle
      """

# Executando o Comando SQL
rs = conn.execute(sql1)

for row in rs:
    print(row)

('German, DMY',)


Vamos verificar se funcionou com uma consulta no SQL:

In [88]:
sql = """
       SELECT '1/8/1999'::DATE;
	  """
rs = conn.execute(sql)
for row in rs:
    print(row)

(datetime.date(1999, 1, 8),)


Como podemos ver, quando inserimos uma string, '1/8/1999', usando o formato **Mês, Dia, Ano,** o *Postgres* entende que é 8 de janeiro de 1999 (e não 1º de agosto de 1999). Exibe a data usando o formato ISO especificado anteriormente, no formato AAAA-MM-DD.

Da mesma forma, podemos usar os seguintes formatos com traços e pontos para separar os componentes de data:

In [89]:
sql = """
      SELECT '1-8-1999'::DATE;
	  """
rs = conn.execute(sql)
for row in rs:
    print(row)

(datetime.date(1999, 1, 8),)


In [90]:
sql = """
      SELECT '1.8.1999'::DATE;
	  """
rs = conn.execute(sql)
for row in rs:
    print(row)

(datetime.date(1999, 1, 8),)


Além de exibir datas que são inseridas como strings, podemos exibir a data atual de maneira muito simples usando as palavras-chave **current_date** no Postgres:

In [91]:
sql = """
      SELECT current_date;
	  """
rs = conn.execute(sql)
for row in rs:
    print(row)

(datetime.date(2022, 7, 11),)


Além do tipo de dados **DATE**, o padrão SQL oferece um tipo de dado **TIMESTAMP**. Um carimbo de data/hora representa uma data e uma hora, até um microssegundo.

Podemos ver o timestamp atual usando a função **now()** e podemos especificar o nosso fuso horário usando **AT TIME ZONE 'UTC'**. Aqui está um exemplo da função **now()** com o fuso horário Eastern Standard especificado:

In [93]:
sql = """
      SELECT now() AT TIME ZONE 'EST';
	  """
rs = conn.execute(sql)
for row in rs:
    print(row)

(datetime.datetime(2022, 7, 11, 19, 43, 15, 464824),)


Também podemos usar o tipo de dados timestamp sem o fuso horário especificado. Você pode pegar o fuso horário atual com a função **now()**:

In [94]:
sql = """
      SELECT now();
	  """
rs = conn.execute(sql)
for row in rs:
    print(row)

(datetime.datetime(2022, 7, 11, 20, 43, 15, 464824, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000))),)


> #### Observação
> Em geral, é recomendável usar um carimbo de data/hora com o fuso horário especificado. Se você não especificar o fuso horário, o valor do carimbo de data/hora pode ser questionável (por exemplo, o horário pode ser representado no fuso horário em que a empresa está localizada, no horário Universal Time Coordinated (UTC) ou no fuso horário do cliente ).

Os tipos de dados date e timestamp são úteis não apenas porque exibem datas em um formato legível, mas também porque armazenam esses valores usando menos bytes do que a representação de string equivalente (um valor de tipo de data requer apenas 4 bytes, enquanto a representação de texto equivalente pode ser 8 bytes para uma representação de 8 caracteres, como **'20160101'**). Além disso, o *Postgres* fornece funcionalidades especiais para manipular e transformar datas, e isso é particularmente útil para análise de dados.

### Transformando Tipos de Data
Muitas vezes, queremos decompor nossas datas em suas partes componentes. Por exemplo, podemos estar interessados apenas no ano e no mês, mas não no dia, para a análise mensal de nossos dados. Para fazer isso, podemos usar **EXTRACT(component FROM data)**. Aqui está um exemplo:

In [98]:
import pandas as pd

sql = """
      SELECT current_date,
        EXTRACT(year FROM current_date) AS year,
        EXTRACT(month FROM current_date) AS month,
        EXTRACT(day FROM current_date) AS day;
	  """
pd.read_sql_query(con=conn, sql=sql)

,current_date,year,month,day
0,2022-07-11,2022.0,7.0,11.0


Da mesma forma, podemos abreviar esses componentes como **y**, **mon** e **d**, e o *Postgres* entenderá o que queremos:

In [99]:
sql = """
      SELECT current_date,
            EXTRACT(y FROM current_date) AS year,
            EXTRACT(mon FROM current_date) AS month,
            EXTRACT(d FROM current_date) AS day;
	  """
pd.read_sql_query(con=conn, sql=sql)

,current_date,year,month,day
0,2022-07-11,2022.0,7.0,11.0


Além do ano, mês e dia, às vezes queremos componentes adicionais, como dia da semana, semana do ano ou trimestre. Você também pode extrair essas partes de data da seguinte forma:

In [100]:
sql = """
      SELECT current_date,
        EXTRACT(dow FROM current_date) AS day_of_week,
        EXTRACT(week FROM current_date) AS week_of_year,
        EXTRACT(quarter FROM current_date) AS quarter;
	  """
pd.read_sql_query(con=conn, sql=sql)

,current_date,day_of_week,week_of_year,quarter
0,2022-07-11,1.0,28.0,3.0


Observe que **EXTRACT** sempre gera um número, portanto, neste caso, **day_of_week** começa em 0 (domingo) e vai até 6 (sábado). Em vez de **dow**, você pode usar **isodow**, que começa em 1 (segunda-feira) e vai até 7 (domingo).

Além de extrair partes de uma data, podemos simplesmente truncar nossa data ou carimbo de data/hora. Por exemplo, podemos querer simplesmente truncar nossa data para o ano e mês. Podemos fazer isso usando a função **DATE_TRUNC()**:

In [101]:
sql = """
      SELECT NOW(), DATE_TRUNC('month', NOW());
	  """
pd.read_sql_query(con=conn, sql=sql)

,now,date_trunc
0,2022-07-12 00:43:15.464824+00:00,2022-07-01 04:00:00+00:00


Observe que a função **DATE_TRUNC (...)** não arredonda o valor. Em vez disso, ele gera o maior valor arredondado menor ou igual ao valor de data inserido.
> ##### Observação
> A função **DATE_TRUNC(...)** é semelhante à função de *floor* em matemática, que gera o maior inteiro menor ou igual à entrada (por exemplo, 5.7 seria reduzido a 5).

A função **DATE_TRUNC (...)** é particularmente útil para instruções **GROUP BY**. Por exemplo, você pode usá-lo para agrupar as vendas por trimestre e obter o total de vendas trimestrais:

In [102]:
sql = """
      SELECT DATE_TRUNC('quarter', NOW()) AS quarter,
            SUM(sales_amount) AS total_quarterly_sales
      FROM sales
      GROUP BY 1
      ORDER BY 1 DESC;
	  """
pd.read_sql_query(con=conn, sql=sql)

,quarter,total_quarterly_sales
0,2022-07-01 04:00:00+00:00,2.672534e+08


> ##### Observação
> **DATE_TRUNC(...)** requer uma string representando o campo para o qual você deseja truncar, enquanto **EXTRACT(...)** aceita a representação de string (com aspas) ou o nome do campo (sem aspas).

### Intervalos
Além de representar datas, também podemos representar intervalos de tempo fixos usando o tipo de dados **INTERVAL**. Isso é útil se quisermos analisar quanto tempo demora algo, por exemplo, se quisermos saber quanto tempo um cliente leva para fazer uma compra.

Aqui está um exemplo:

In [103]:
sql = """
      SELECT INTERVAL '5 days';
	  """
pd.read_sql_query(con=conn, sql=sql)

,interval
0,5 days


Os intervalos são úteis para subtrair *timestamps* de data/hora, por exemplo:

In [104]:
sql = """
      SELECT TIMESTAMP '2016-03-01 00:00:00' - TIMESTAMP '2016-02-01 00:00:00'
        AS days_in_feb;
	  """
pd.read_sql_query(con=conn, sql=sql)

,days_in_feb
0,29 days


Ou, alternativamente, os intervalos podem ser usados para adicionar o número de dias a um *timestamps* de data/hora:

In [105]:
sql = """
      SELECT TIMESTAMP '2016-03-01 00:00:00' + INTERVAL '7 days' AS new_date;
	  """
pd.read_sql_query(con=conn, sql=sql)

,new_date
0,2016-03-08


Embora os intervalos ofereçam um método preciso para fazer aritmética de *timestamps* de data/hora, o formato **DATE** pode ser usado com números inteiros para obter um resultado semelhante. No exemplo a seguir, simplesmente adicionamos 7 (um inteiro) à data para calcular a nova data:

In [106]:
sql = """
      SELECT DATE '2016-03-01' + 7 AS new_date;
	  """
pd.read_sql_query(con=conn, sql=sql)

,new_date
0,2016-03-08


Da mesma forma, podemos subtrair duas datas e obter um resultado inteiro:

In [107]:
sql = """
      SELECT DATE '2016-03-01' - DATE '2016-02-01' AS days_in_feb;
	  """
pd.read_sql_query(con=conn, sql=sql)

,days_in_feb
0,29


Enquanto o tipo de dados de data oferece facilidade de uso, o *timestamp* com o tipo de dados de fuso horário oferece precisão. Se você precisar que seu campo de data/hora seja exatamente igual à hora em que a ação ocorreu, use o *timestamp* com o fuso horário. Caso contrário, você pode usar o campo de data.